Kaggle Competition Practice: Digit Recognizer

You will build DNN to compete with worldwide teams.

https://www.kaggle.com/c/digit-recognizer

SoumyaSharma102
Score: 0.99242

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Sequential,Model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import keras
from keras import backend as K

In [3]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y_train = train['label'].astype('float32')
X_train = train.drop(['label'], axis=1).astype('int32')
X_test = test.astype('float32')
X_train.shape, y_train.shape, X_test.shape

((42000, 784), (42000,), (28000, 784))

In [5]:
X_train = X_train/255
X_test = X_test/255

X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

y_train = to_categorical(y_train, num_classes = 10)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state=42)

In [7]:
input_shape = (28,28,1)
X_input = Input(input_shape)

# layer 1
x = Conv2D(64,(3,3),strides=(1,1),name='layer_conv1',padding='same')(X_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool1')(x)
# layer 2
x = Conv2D(32,(3,3),strides=(1,1),name='layer_conv2',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool2')(x)
# layer 3
x = Conv2D(32,(3,3),strides=(1,1),name='conv3',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2), name='maxPool3')(x)
# fc
x = Flatten()(x)
x = Dense(64,activation ='relu',name='fc0')(x)
x = Dropout(0.25)(x)
x = Dense(32,activation ='relu',name='fc1')(x)
x = Dropout(0.25)(x)
x = Dense(10,activation ='softmax',name='fc2')(x)

model_A = Model(inputs=X_input, outputs=x, name='Predict')
model_A.summary()

Model: "Predict"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
layer_conv1 (Conv2D)         (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 64)        0         
_________________________________________________________________
maxPool1 (MaxPooling2D)      (None, 14, 14, 64)        0         
_________________________________________________________________
layer_conv2 (Conv2D)         (None, 14, 14, 32)        18464     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128 

2022-03-13 19:51:20.328085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 19:51:20.422383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 19:51:20.423115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 19:51:20.424323: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
model_A.compile(optimizer='Adagrad',loss='categorical_crossentropy',metrics=['accuracy'])
model_A.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_valid, y_valid))

2022-03-13 19:51:22.884556: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-03-13 19:51:24.285850: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


378/378 [==============================] - 9s 6ms/step - loss: 1.8001 - accuracy: 0.3793 - val_loss: 1.9774 - val_accuracy: 0.3667
Epoch 2/30
378/378 [==============================] - 2s 5ms/step - loss: 1.2106 - accuracy: 0.6011 - val_loss: 0.7610 - val_accuracy: 0.8643
Epoch 3/30
378/378 [==============================] - 2s 5ms/step - loss: 0.9395 - accuracy: 0.7025 - val_loss: 0.5190 - val_accuracy: 0.9062
Epoch 4/30
378/378 [==============================] - 2s 5ms/step - loss: 0.7759 - accuracy: 0.7604 - val_loss: 0.3975 - val_accuracy: 0.9248
Epoch 5/30
378/378 [==============================] - 2s 5ms/step - loss: 0.6748 - accuracy: 0.7943 - val_loss: 0.3315 - val_accuracy: 0.9352
Epoch 6/30
378/378 [==============================] - 2s 5ms/step - loss: 0.5946 - accuracy: 0.8237 - val_loss: 0.2804 - val_accuracy: 0.9443
Epoch 7/30
378/378 [==============================] - 2s 5ms/step - loss: 0.5416 - accuracy: 0.8371 - val_loss: 0.2457 - val_accuracy: 0.9474
Epoch 8/30
378/37

In [9]:
model_A.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_A.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_valid, y_valid))

Epoch 1/30
378/378 [==============================] - 3s 6ms/step - loss: 0.1758 - accuracy: 0.9517 - val_loss: 0.0708 - val_accuracy: 0.9800
Epoch 2/30
378/378 [==============================] - 2s 5ms/step - loss: 0.1013 - accuracy: 0.9726 - val_loss: 0.0592 - val_accuracy: 0.9838
Epoch 3/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0803 - accuracy: 0.9782 - val_loss: 0.0839 - val_accuracy: 0.9762
Epoch 4/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0658 - accuracy: 0.9823 - val_loss: 0.0711 - val_accuracy: 0.9810
Epoch 5/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0561 - accuracy: 0.9839 - val_loss: 0.0408 - val_accuracy: 0.9862
Epoch 6/30
378/378 [==============================] - 2s 6ms/step - loss: 0.0541 - accuracy: 0.9857 - val_loss: 0.0496 - val_accuracy: 0.9857
Epoch 7/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0409 - accuracy: 0.9884 - val_loss: 0.0492 - val_accuracy: 0.9855
Epoch 

In [10]:
model_A.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
model_A.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_valid, y_valid))

Epoch 1/30
378/378 [==============================] - 3s 5ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0507 - val_accuracy: 0.9905
Epoch 2/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 0.0506 - val_accuracy: 0.9902
Epoch 3/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0042 - accuracy: 0.9989 - val_loss: 0.0492 - val_accuracy: 0.9907
Epoch 4/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.0489 - val_accuracy: 0.9910
Epoch 5/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0038 - accuracy: 0.9991 - val_loss: 0.0504 - val_accuracy: 0.9902
Epoch 6/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0041 - accuracy: 0.9988 - val_loss: 0.0497 - val_accuracy: 0.9907
Epoch 7/30
378/378 [==============================] - 2s 5ms/step - loss: 0.0036 - accuracy: 0.9990 - val_loss: 0.0481 - val_accuracy: 0.9912
Epoch 

In [11]:
model_A.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
model_A.fit(X_train, y_train, epochs=20, batch_size=100, validation_data=(X_valid, y_valid))

Epoch 1/20
378/378 [==============================] - 3s 5ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.0554 - val_accuracy: 0.9912
Epoch 2/20
378/378 [==============================] - 2s 4ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 0.0552 - val_accuracy: 0.9917
Epoch 3/20
378/378 [==============================] - 2s 4ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.0550 - val_accuracy: 0.9919
Epoch 4/20
378/378 [==============================] - 2s 5ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.0554 - val_accuracy: 0.9917
Epoch 5/20
378/378 [==============================] - 2s 4ms/step - loss: 0.0029 - accuracy: 0.9993 - val_loss: 0.0556 - val_accuracy: 0.9914
Epoch 6/20
378/378 [==============================] - 2s 5ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0565 - val_accuracy: 0.9914
Epoch 7/20
378/378 [==============================] - 2s 5ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0564 - val_accuracy: 0.9914
Epoch 

In [12]:
y_pred = model_A.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)

submission = pd.DataFrame({'ImageId': list(range(1, len(y_pred)+1)), 'Label': y_pred})
submission.to_csv("./submission.csv",index=False)